In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob

print(datetime.datetime.now())
os.getcwd()

2020-02-13 17:30:33.426879


'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Planner_Request/JT_product_mapping_and_YoY_20200212'

In [2]:
df_prod_taxonomy=pd.read_csv("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20200201-134613-815.txt",
                            sep='|',dtype=str)

df_div_name=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",
                            sep='|',dtype=str)

df_department_name=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Department Names.txt",
                            sep='|',dtype=str)

df_class_name=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Class Names.txt",
                            sep='|',dtype=str,encoding="iso-8859-1")


In [3]:
df_prod_taxonomy=pd.merge(df_prod_taxonomy,df_div_name,on="division_id",how="left")
df_prod_taxonomy=pd.merge(df_prod_taxonomy,df_department_name,on="department_id",how="left")
df_prod_taxonomy=pd.merge(df_prod_taxonomy,df_class_name,on="class_code_id",how="left")
df_prod_taxonomy=df_prod_taxonomy[['division_id','division_desc','department_id','department_desc',
                                  'class_code_id','class_code_desc','subclass_id','subclass_desc']]

In [4]:
df_prod_taxonomy.head(2)

,division_id,division_desc,department_id,department_desc,class_code_id,class_code_desc,subclass_id,subclass_desc
0,1,FOOD,108,CANDYSNACK,11001,CANDY,1,Chocolate Bagged
1,1,FOOD,108,CANDYSNACK,11001,CANDY,3,Single Chocolate


In [5]:
df_1=df_prod_taxonomy[df_prod_taxonomy['department_desc']=="CANSNPASTA"]
df_1[['division_id','department_id']].drop_duplicates()

,division_id,department_id
17,1,109


In [6]:
df_2=df_prod_taxonomy[df_prod_taxonomy['department_desc']=="CHRISTMAS"]
df_2[['division_id','department_id']].drop_duplicates()

,division_id,department_id
435,3,360


In [7]:
df_3=df_prod_taxonomy[df_prod_taxonomy['department_desc']=="ELECTRONICS"]
df_3[['division_id','department_id']].drop_duplicates()

,division_id,department_id
2159,6,550


In [8]:
df_4=df_prod_taxonomy[df_prod_taxonomy['department_desc']=="STATIONERY"]
df_4[['division_id','department_id']].drop_duplicates()

,division_id,department_id
1962,5,520


In [12]:
df_products_wanted=df_1.append(df_2).append(df_3).append(df_4)
print(df_products_wanted.shape)
df_products_wanted[['division_id','department_id','division_desc','department_desc']].drop_duplicates()

(305, 8)


,division_id,department_id,division_desc,department_desc
17,1,109,FOOD,CANSNPASTA
435,3,360,SEASONAL,CHRISTMAS
2159,6,550,ELECNACCES,ELECTRONICS
1962,5,520,HARDHOME,STATIONERY


In [13]:
list_files_2018Q4=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
list_files_2018Q4=[x for x in list_files_2018Q4 if x.split("d/MediaStormDailySalesHistory")[1][:8]>="20181104"]
list_files_2018Q4=[x for x in list_files_2018Q4 if x.split("d/MediaStormDailySalesHistory")[1][:8]<="20190202"]
list_files_2018Q4.sort()
print(len(list_files_2018Q4))
# list_files_2018Q4

13


In [14]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
list_files_2019Q4=list(recursive_file_gen("/home/jian/BigLots/"))
list_files_2019Q4=[x for x in list_files_2019Q4 if "/MediaStorm_" in x and "daily" in x.lower()]
list_files_2019Q4=[x for x in list_files_2019Q4 if x.split("/MediaStorm_")[1][:10]>="2019-11-03"]
list_files_2019Q4=[x for x in list_files_2019Q4 if x.split("/MediaStorm_")[1][:10]<="2020-02-02"]
list_files_2019Q4.sort()

In [15]:
# 2018 sales by class
df_2018_wanted=pd.DataFrame()
for file in list_files_2018Q4:
    df=pd.read_csv(file,dtype=str,sep="|")
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['item_transaction_units']=df['item_transaction_units'].astype(int)
    df=pd.merge(df,df_products_wanted,on=['class_code_id','subclass_id'],how="left")
    df=df[pd.notnull(df['department_id'])]
    df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"nonrewards","rewards")
    
    df_trans=df[['location_id','transaction_dt','transaction_id','rewards_label','customer_id_hashed','division_id','department_id','class_code_id']].drop_duplicates()
    df_trans['trans']=1
    df_trans=df_trans.groupby(['location_id','transaction_dt','transaction_id','rewards_label','division_id','department_id','class_code_id'])['trans'].sum().to_frame().reset_index()
    df_sales=df.groupby(['location_id','transaction_dt','transaction_id','rewards_label','division_id','department_id','class_code_id'])['item_transaction_amt','item_transaction_units'].sum().reset_index()
    df_sales=df_sales.rename(columns={"item_transaction_amt":"sales",'item_transaction_units':"units"})
    
    df=pd.merge(df_sales,df_trans,on=['location_id','transaction_dt','transaction_id','rewards_label','division_id','department_id','class_code_id'],how="outer")
    df_2018_wanted=df_2018_wanted.append(df)
    print(datetime.datetime.now(),df['transaction_dt'].min())

2020-02-13 17:34:48.654550 2018-11-04
2020-02-13 17:36:51.091276 2018-11-11
2020-02-13 17:40:34.022891 2018-11-18
2020-02-13 17:43:48.815483 2018-11-25
2020-02-13 17:47:36.200241 2018-12-02
2020-02-13 17:51:38.861568 2018-12-09
2020-02-13 17:56:06.679127 2018-12-16
2020-02-13 17:57:46.980103 2018-12-23
2020-02-13 17:58:47.787632 2018-12-30
2020-02-13 17:59:49.237723 2019-01-06
2020-02-13 18:01:11.110800 2019-01-13
2020-02-13 18:02:52.211270 2019-01-20
2020-02-13 18:03:46.704769 2019-01-27


In [16]:
# 2019 sales by class
df_2019_wanted=pd.DataFrame()
for file in list_files_2019Q4:
    df=pd.read_csv(file,dtype=str,sep="|")
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['item_transaction_units']=df['item_transaction_units'].astype(int)
    df=pd.merge(df,df_products_wanted,on=['class_code_id','subclass_id'],how="left")
    df=df[pd.notnull(df['department_id'])]
    df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"nonrewards","rewards")
    
    df_trans=df[['location_id','transaction_dt','transaction_id','rewards_label','customer_id_hashed','division_id','department_id','class_code_id']].drop_duplicates()
    df_trans['trans']=1
    df_trans=df_trans.groupby(['location_id','transaction_dt','transaction_id','rewards_label','division_id','department_id','class_code_id'])['trans'].sum().to_frame().reset_index()
    df_sales=df.groupby(['location_id','transaction_dt','transaction_id','rewards_label','division_id','department_id','class_code_id'])['item_transaction_amt','item_transaction_units'].sum().reset_index()
    df_sales=df_sales.rename(columns={"item_transaction_amt":"sales",'item_transaction_units':"units"})
    
    df=pd.merge(df_sales,df_trans,on=['location_id','transaction_dt','transaction_id','rewards_label','division_id','department_id','class_code_id'],how="outer")
    df_2019_wanted=df_2019_wanted.append(df)
    print(datetime.datetime.now(),df['transaction_dt'].min())

2020-02-13 18:05:40.975900 2019-11-03
2020-02-13 18:07:39.391115 2019-11-10
2020-02-13 18:09:01.666213 2019-11-17
2020-02-13 18:11:48.420997 2019-11-24
2020-02-13 18:14:29.363060 2019-12-01
2020-02-13 18:17:28.758909 2019-12-08
2020-02-13 18:20:26.877198 2019-12-15
2020-02-13 18:22:18.632780 2019-12-22
2020-02-13 18:23:40.326415 2019-12-29
2020-02-13 18:24:38.601110 2020-01-05
2020-02-13 18:25:52.714199 2020-01-12
2020-02-13 18:27:04.796886 2020-01-19
2020-02-13 18:27:54.788268 2020-01-26


In [17]:
print("df_2019_wanted.shape",df_2019_wanted.shape)
print(df_2019_wanted['transaction_dt'].min(),df_2019_wanted['transaction_dt'].max())

print("df_2018_wanted.shape",df_2018_wanted.shape)
print(df_2018_wanted['transaction_dt'].min(),df_2018_wanted['transaction_dt'].max())


df_2019_wanted.shape (24820466, 10)
2019-11-03 2020-02-01
df_2018_wanted.shape (29554978, 10)
2018-11-04 2019-02-02


In [18]:
df_output_2019Q4=df_2019_wanted.groupby(['division_id','department_id','class_code_id','rewards_label'])['sales','units','trans'].sum().reset_index()
df_output_2019Q4['Quarter']="2019Q4"

df_output_2018Q4=df_2018_wanted.groupby(['division_id','department_id','class_code_id','rewards_label'])['sales','units','trans'].sum().reset_index()
df_output_2018Q4['Quarter']="2018Q4"

In [27]:
df_naming=df_prod_taxonomy[['division_id','department_id','class_code_id',
                           'division_desc','department_desc','class_code_desc']].drop_duplicates()
df_output_2019Q4=pd.merge(df_output_2019Q4,df_naming,on=['division_id','department_id','class_code_id'],how="left")
df_output_2018Q4=pd.merge(df_output_2018Q4,df_naming,on=['division_id','department_id','class_code_id'],how="left")




In [29]:
writer=pd.ExcelWriter("./BL_YoY_selected_dropping_prods_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_2019Q4.to_excel(writer,"Q4_2019",index=False)
df_output_2018Q4.to_excel(writer,"Q4_2018",index=False)
writer.save()

In [30]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Planner_Request/JT_product_mapping_and_YoY_20200212'